# Coursera Capstone Project The Battle of Neighborhood
This is the final project in the IBM Data Science course in Coursera. In this project the student should realize an own idea with geo location information.
## Idea of this notebook
In this notebook it should be searched the best neighborhood of Toronto to live as a bicyclist. For this reason it will be used the data from the last week to explore the location in Toronto. To get bicycle related data it should be used Openstreetmap to scrap data.
## What are bicycle related?
The first step is to determine which information can be used
* length of bicycle lanes
* amount of public parks
* amount of destinations for cyclist
* amount of bicycle workshops, shops for spare parts
* squarefeet of puplic parks

## Getting data
The data should be got from Open Street Map (OSM). This is a open source project and there are several APIs to get information.
### 1. Get borough locations
From Open Street Map I download the outer bounds of the boroughs of Toronto.

Install software to download from OSM. The OSMPythonTools are
* Overpass is a tool make queries on the [Overpass](https://wiki.openstreetmap.org/wiki/Overpass_API)
* [Moinatim](https://wiki.openstreetmap.org/wiki/Nominatim) is to geo codes from OSM
* there are other tools included which are not used in this notebook

In [39]:
!pip3 install OSMPythonTools

In [40]:
import pandas as pd
import numpy as np
import json
import copy
from OSMPythonTools.overpass import Overpass
ovp = Overpass()
from OSMPythonTools.api import Api
api = Api()
import folium # map rendering library

#### Sorting the Data
To show the bounderies of the boroughs correct, it is needed to sort them after download.

In [41]:
def SortNode(ways):
    index = 0
    sortidx = 0
    while sortidx < (len(ways) - 1):
        ways_idx = ways[sortidx + 1].copy()
        ways_idx_rev = ways[sortidx + 1].copy()
        ways_idx_rev.reverse()
        #print(f'sortidx: {sortidx}\r\n')   
        #print(f'ways[{sortidx}]: {ways[sortidx]}\r\nways[{sortidx + 1}]: {ways[sortidx + 1]}\r\n')
        #print(f'ways_idx: {ways_idx}\r\nways_idx_rev: {ways_idx_rev}\r\n')
        #print(f'ways[{sortidx}][-1]: {ways[sortidx][-1]}\r\nways_idx[0]: {ways_idx[0]}\r\n')
        #print(f'ways[{sortidx}][-1]: {ways[sortidx][-1]}\r\nways_idx_rev[0]: {ways_idx_rev[0]}\r\n')
        if ways[sortidx][-1] == ways_idx[0]:
            sortidx = sortidx + 1
        elif ways[sortidx][-1] == ways_idx_rev[0]:
            ways[sortidx + 1] = ways_idx_rev
            sortidx = sortidx + 1
        else:
            index = sortidx + 2
            if index >= len(ways):
                index = 0
            while index != 1:
                ways_idx = ways[index].copy()
                ways_idx_rev = ways[index].copy()
                ways_idx_rev.reverse()
                #print(f'index: {index}\r\n')
                #print(f'ways[{sortidx}]: {ways[sortidx]}\r\nways[{index}]: {ways[index]}\r\n')
                #print(f'ways_idx: {ways_idx}\r\n')
                #print(f'ways_idx_rev: {ways_idx_rev}\r\n')
                #print(f'ways[{sortidx}][-1]: {ways[sortidx][-1]}\r\nways_idx[0]: {ways_idx[0]}\r\n')
                #print(f'ways[{sortidx}][-1]: {ways[sortidx][-1]}\r\nways_idx_rev[0]: {ways_idx_rev[0]}\r\n')
                if ways[sortidx][-1] == ways_idx[0]:
                    ways.pop(index);
                    ways.insert(sortidx + 1, ways_idx)
                    sortidx = 0
                    index = 1
                    break
                elif ways[sortidx][-1] == ways_idx_rev[0]:
                    ways.pop(index);
                    ways.insert(sortidx + 1, ways_idx_rev)
                    sortidx = 0
                    index = 1
                    break
                else:
                    index = index + 1
                    if index >= len(ways):
                        index = 0
            sortidx = sortidx + 1

    #print(f'ways[0]: {ways[0]}\r\nways[1]: {ways[1]}\r\n')
    #print(f'ways after: {ways}\r\n')
    return ways

def OverpassQuery(query):
    result_w = ovp.query(query, timeout=100)
    res_json = result_w.toJSON()
    res_elements = res_json['elements']
    #print(f'res_json:\r\n{res_json}\r\n')
    #print(f'type(res_json):\r\n{type(res_json)}\r\n')
    #print(f'res_json.keys:\r\n{res_json.keys()}\r\n')
    return res_elements

In [54]:
#search_query = f'relation["boundary"="administrative"]["name"="Don Valley West"];way(r);out qt;'
borough = ["Scarborough", "North York", "Old Toronto", "Etobicoke", "York", "East York"]

Scarborough_query = f'area[name="Toronto"];relation["boundary"="administrative"]["name"="{borough[0]}"](area);way(r);out qt;'
North_York_query = f'area[name="Toronto"];relation["boundary"="administrative"]["name"="{borough[1]}"](area);way(r);out qt;'
Old_Toronto_query = f'area[name="Toronto"];relation["boundary"="administrative"]["name"="{borough[2]}"](area);way(r);out qt;'
Etobicoke_query = f'area[name="Toronto"];relation["boundary"="administrative"]["name"="{borough[3]}"](area);way(r);out qt;'
York_query = f'area[name="Toronto"];relation["boundary"="administrative"]["name"="{borough[4]}"](area);way(r);out qt;'
East_Yorkh_query = f'area[name="Toronto"];relation["boundary"="administrative"]["name"="{borough[5]}"](area);way(r);out qt;'
#search_query = f'relation["boundary"="administrative"]["name"="Don Valley West"];way(r);out geom qt;'
#search_query = f'relation["boundary"="administrative"]["name"="Don Valley West"];out qt;>;out skel qt;'

queries = [Scarborough_query, North_York_query, Old_Toronto_query, Etobicoke_query, York_query, East_Yorkh_query]

overpass_response = []
for x in queries:
    tmp = OverpassQuery(x).copy()
    overpass_response.append(tmp)

ways = []
for y in overpass_response:
    #print(f'y: {y}\r\n')
    #print(f'new_overpass_query\r\n')

    ways_inner = []
    for x in y:
        #print(f'x: {x}\r\n')
        ways_inner.append(x['nodes'])

    ways.append(ways_inner)



for x in ways:
    #print(f'ways(x) before: {x}\r\n')
    x = SortNode(x)
    #print(f'ways(x) after: {x}\r\n')

ways_coordinates = []
node_list = []
bound_lat_max = -360
bound_lat_min = 360
bound_lon_max = -360
bound_lon_min = 360
for bor in ways:
    ways_coordinates_bor = []
    node_list_inner = []
    for w in bor:        
        ways_coordinates_inner = []
        for x in w:
            #print(f'x: {x}\r\n')
            #print(f'api.query({x})\r\n')
            result_ways = api.query(f'node/{x}')
            ways_coordinates_inner.append([result_ways.lat(),result_ways.lon()])
            node_list_inner.append([result_ways.lat(),result_ways.lon()])
            if result_ways.lat() > bound_lat_max:
                bound_lat_max = result_ways.lat()
            if result_ways.lat() < bound_lat_min:
                bound_lat_min = result_ways.lat()
            if result_ways.lon() > bound_lon_max:
                bound_lon_max = result_ways.lon()
            if result_ways.lon() < bound_lon_min:
                bound_lon_min = result_ways.lon()
        ways_coordinates_bor.append(ways_coordinates_inner)
    node_list.append(node_list_inner)

    ways_coordinates.append(ways_coordinates_bor)
    #print(f'ways_cor: {ways_cor}\r\n')
    #print(f'type(ways_cor): {type(ways_cor_inner)}\r\n')
print(f'bound_lat_min: {bound_lat_min}, bound_lon_min: {bound_lon_min}, bound_lat_max: {bound_lat_max}, bound_lon_max: {bound_lon_max}')

node_list_reverse = []
#print(f'node_list_reverse: {node_list_reverse}')

#print(f'node_list: {node_list[0]}')

for x in node_list:
    tmp_inner = []
    for i in x:
        tmp_inner.append([i[1], i[0]])
    node_list_reverse.append(tmp_inner)

#print(f'node_list: {node_list}')
#print(f'node_list_reverse: {node_list_reverse}')


bound_lat_min: 43.5802533, bound_lon_min: -79.6392727, bound_lat_max: 43.8554425, bound_lon_max: -79.1132193


#### Visualize the boroughs in Folium
To dexcribe the doing better it helps to view the boroughs in a map. The map are colored withe bounderies and areas of the boroughs.
The bounderies are used to get the points to compare the boroughs in respect to bicycle interesting topics.

In [55]:
latitude = 43.7134408
longitude = -79.541716

#color_map = [0x800000, 0x8B0000, 0xA52A2A, 0xB22222, 0xDC143C, 0xFF0000, 0xFF6347, 0xFF7F50, 0xCD5C5C, 
#           0xF08080, 0xE9967A, 0xFA8072, 0xFFA07A, 0xFF4500, 0xFF8C00, 0xFFA500, 0xFFD700]
#color_map = [0x800000, 0xA52A2A, 0xDC143C, 0xFF6347, 0xCD5C5C, 0xE9967A, 0xFFA07A, 0xFF8C00, 0xFFD700]

map_select = folium.Map(location=[latitude, longitude], zoom_start=10)
color_map = ['red', 'green', 'blue', 'yellow', 'violet', 'black']
#integer color value = R + G*(256) + B*(256^2)
#color_map = list(range(134, 255, 24))
    
for x, col in zip(node_list, color_map):
    #folium.PolyLine(x, fill = True, fill_color=f'#{col:06x}', fill_opacity =0.6).add_to(map_select)
    folium.PolyLine(x, fill = True, fill_color=col, fill_opacity =0.6).add_to(map_select)

#folium.LayerControl().add_to(map_select)

# add markers to map
#for lat, lng, borough, neighborhood in zip(tmp['Latitude'], tmp['Longitude'], tmp['Borough'], tmp['Neighbourhood']):
#    label = '{}, {}'.format(neighborhood, borough)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=5,
#        popup=label,
#        color='blue',
#        fill=True,
#        fill_color='#3186cc',
#        fill_opacity=0.7,
#        parse_html=False).add_to(map_select)  
#m.fit_bounds([[52.193636, -2.221575], [52.636878, -1.139759]])
#map_select.fit_bounds([[latitude - bbox_off, longitude - bbox_off], [latitude + bbox_off, longitude + bbox_off]]) 
print(f'bound_lat_min: {bound_lat_min}, bound_lon_min: {bound_lon_min}, bound_lat_max: {bound_lat_max}, bound_lon_max: {bound_lon_max}')
map_select.fit_bounds([[bound_lat_min, bound_lon_min], [bound_lat_max, bound_lon_max]]) 
map_select

bound_lat_min: 43.5802533, bound_lon_min: -79.6392727, bound_lat_max: 43.8554425, bound_lon_max: -79.1132193


Boroughs of Toronto:
    
|Borough | Color |
| :-- | --- | 
|Scarborough | Red |
|North York | green |
|Old Toronto | Blue |
|Etobicoke | Yellow |
|York | Violet |
|East York | Black |

### 2. Get bicycle lanes and tracks
To measure the length of possible biking kilometers or miles we extract the bicycle lanes and track positions from OSM and calculate his length in the boroughs. At first let's look at the bicycle tracks of every borough.

In [56]:
node_list_string = []
for n in node_list:
    node_tmp = f''
    for x in n:
        node_tmp = node_tmp + ' ' + str(x[0]) + ' ' + str(x[1])
        
    node_list_string.append(node_tmp)

borough_ways_geo = []
for b in node_list_string:
    query = f'(way["cycleway"](poly:"{b}");way["bicycle"="yes"](poly:"{b}");\
        way["segregated"](poly:"{b}");way["highway"="cycleway"](poly:"{b}"););\
        out geom qt;'
    
    tmp = OverpassQuery(query)
    
    ways_geo = []
    for w in tmp:
        geo_loc = []
        for n in w["geometry"]:
            #print(f'n: {n["lat"]}')
            geo_loc.append([n["lat"], n["lon"]]) 

        ways_geo.append(geo_loc)
    borough_ways_geo.append(ways_geo)

In [57]:
latitude = 43.7134408
longitude = -79.541716

#color_map = [0x800000, 0x8B0000, 0xA52A2A, 0xB22222, 0xDC143C, 0xFF0000, 0xFF6347, 0xFF7F50, 0xCD5C5C, 
#           0xF08080, 0xE9967A, 0xFA8072, 0xFFA07A, 0xFF4500, 0xFF8C00, 0xFFA500, 0xFFD700]
#color_map = [0x800000, 0xA52A2A, 0xDC143C, 0xFF6347, 0xCD5C5C, 0xE9967A, 0xFFA07A, 0xFF8C00, 0xFFD700]

map_select = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='Openstreetmap',)
color_map = ['red', 'green', 'blue', 'yellow', 'violet', 'black']
#integer color value = R + G*(256) + B*(256^2)
#color_map = list(range(134, 255, 24))

for b,c,n in zip(borough_ways_geo, color_map, node_list):
    for x in b:
        #folium.PolyLine(x, fill = True, fill_color=f'#{col:06x}', fill_opacity =0.6).add_to(map_select)
        folium.PolyLine(x, fill = False, color=c, fill_opacity =0.6).add_to(map_select)
    folium.PolyLine(n, color=c, fill = True, fill_color=c, fill_opacity =0.2).add_to(map_select)

#folium.LayerControl().add_to(map_select)

# add markers to map
#for lat, lng, borough, neighborhood in zip(tmp['Latitude'], tmp['Longitude'], tmp['Borough'], tmp['Neighbourhood']):
#    label = '{}, {}'.format(neighborhood, borough)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=5,
#        popup=label,
#        color='blue',
#        fill=True,
#        fill_color='#3186cc',
#        fill_opacity=0.7,
#        parse_html=False).add_to(map_select)  
#m.fit_bounds([[52.193636, -2.221575], [52.636878, -1.139759]])
#map_select.fit_bounds([[latitude - bbox_off, longitude - bbox_off], [latitude + bbox_off, longitude + bbox_off]]) 
print(f'bound_lat_min: {bound_lat_min}, bound_lon_min: {bound_lon_min}, bound_lat_max: {bound_lat_max}, bound_lon_max: {bound_lon_max}')
map_select.fit_bounds([[bound_lat_min, bound_lon_min], [bound_lat_max, bound_lon_max]]) 
map_select

bound_lat_min: 43.5802533, bound_lon_min: -79.6392727, bound_lat_max: 43.8554425, bound_lon_max: -79.1132193


Here you can see the ways in OSM which marked a for bicycle usage. Every borough is shown in a different color. Now let me show the sum of the length of the ways in the map for each borough.

In [58]:
borough_ways_length = pd.DataFrame([], index=borough, columns=["Number", "Length in km"])
for b, i in zip(node_list_string, borough):
    query_length = f'(way["cycleway"](poly:"{b}");way["bicycle"="yes"](poly:"{b}");\
                     way["segregated"](poly:"{b}");way["highway"="cycleway"](poly:"{b}"););\
                     make statistics number=count(ways), length=sum(length());out;'
    
    tmp = OverpassQuery(query_length)
    borough_ways_length.loc[i] = [tmp[0]["tags"]["number"], float(tmp[0]["tags"]["length"]) / 1000]
borough_ways_length

,Number,Length in km
Scarborough,423,103.757
North York,572,108.545
Old Toronto,1383,221.019
Etobicoke,533,107.204
York,112,24.0808
East York,193,40.0049


In [107]:
#print(f'node_list_reverse: {node_list_reverse}')
geojson = f'{{"type":"FeatureCollection","features":['
for x, b in zip(node_list_reverse, borough):
    geojson = geojson + f'{{"type":"Feature","id":"{b}","properties":{{"name":"{b}"}},\
    "geometry":{{"type":"Polygon","coordinates":[{x}]}}}},'
geojson = geojson[:-1] + f']}}'
#print(geojson)
m = folium.Map(location=[latitude, longitude], zoom_start=10)

folium.Choropleth(
    geo_data=geojson,
    name='choropleth',
    data=borough_ways_length["Length in km"],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Kilometers of bicycle tracks',
    bins = 9
).add_to(m)

folium.LayerControl().add_to(m)
m


In [85]:
borough_data = borough_ways_length.transpose()

borough_data.loc["Area"] = [187.7, 176.9, 97.2, 123.9, 23.2, 21.3]
borough_data.loc["Length per square km"] = borough_data.loc["Length in km"] / borough_data.loc["Area"]
borough_data = borough_data.transpose()
borough_data

,Number,Length in km,Area,Length per square km
Scarborough,423,103.757,187.7,0.552781
North York,572,108.545,176.9,0.613598
Old Toronto,1383,221.019,97.2,2.27386
Etobicoke,533,107.204,123.9,0.865243
York,112,24.0808,23.2,1.03797
East York,193,40.0049,21.3,1.87817


In [106]:
geojson = f'{{"type":"FeatureCollection","features":['
for x, b in zip(node_list_reverse, borough):
    geojson = geojson + f'{{"type":"Feature","id":"{b}","properties":{{"name":"{b}"}},\
    "geometry":{{"type":"Polygon","coordinates":[{x}]}}}},'
geojson = geojson[:-1] + f']}}'
#print(geojson)
m = folium.Map(location=[latitude, longitude], zoom_start=10)
folium.Choropleth(
    geo_data=geojson,
    name='choropleth',
    data=borough_data["Length per square km"],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Length per square km',
    bins = 9
).add_to(m)

folium.LayerControl().add_to(m)
m